In [1]:
import os

In [2]:
%pwd

'd:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_augmentation: bool
    params_image_size: list
    params_batch_size: int
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [5]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_augmentation=self.params.AUGMENTATION,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        
        return prepare_base_model_config

In [7]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras import layers, applications

In [12]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config
    
    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)
    
    def get_base_model(self):
        self.model = applications.InceptionV3(
                        include_top=self.config.params_include_top,
                        weights=self.config.params_weights,
                        input_shape=self.config.params_image_size,
                    )
        
        create_directories([self.config.root_dir])
        
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes, learning_rate, image_size_int, input_shape, trainable=False, augmentation=False):
        if trainable:
            model.trainable = True
        else:
            model.trainable = False
        
        full_model = tf.keras.Sequential()
        
        if augmentation:
            full_model.add(layers.Resizing(image_size_int, image_size_int))
            full_model.add(layers.Rescaling(1.0/image_size_int))
        
        full_model.add(model)
        full_model.add(layers.GlobalAveragePooling2D())
        full_model.add(layers.Flatten())
        full_model.add(layers.Dense(units=208, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)))
        full_model.add(layers.Dropout(0.3))
        full_model.add(layers.Dense(units=208, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)))
        full_model.add(layers.Dropout(0.3))
        full_model.add(layers.Dense(units=208, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)))
        full_model.add(layers.Dropout(0.2))
        full_model.add(layers.Dense(units=classes, activation='softmax'))

        full_model.build(input_shape = input_shape)

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate,
            image_size_int=self.config.params_image_size[0],
            input_shape=[self.config.params_batch_size] + self.config.params_image_size,
            trainable=True,
            augmentation=self.config.params_augmentation
        )
        
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

In [13]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-07-03 16:21:04,738: INFO: common: yaml file config\config.yaml loadded successfully]
[2024-07-03 16:21:04,740: INFO: common: yaml file params.yaml loadded successfully]
[2024-07-03 16:21:04,741: INFO: common: Created directory at: ['artifacts']]
[2024-07-03 16:21:06,479: INFO: common: Created directory at: [WindowsPath('artifacts/prepare_base_model')]]


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (16, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (16, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (16, 6, 6, 2048)       │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (16, 2048)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (16, 2048)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (16, 208)              │       426,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (16, 208)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (16, 208)              │        43,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (16, 208)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (16, 208)              │        43,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (16, 208)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (16, 2)                │           418 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,316,338 (85.13 MB)

 Trainable params: 22,281,906 (85.00 MB)

 Non-trainable params: 34,432 (134.50 KB)